In [10]:

import os
from typing import List, Dict


from omegaconf import DictConfig, OmegaConf
import torch
from flp2p.client import FLClient
from flp2p.data import build_client_loaders, get_dataset
from flp2p.graph_runner import run_rounds
from flp2p.networks.lenet5 import LeNet5
from flp2p.networks.resnet18 import make_resnet18
import logging
import pickle
import random
import numpy as np
import torch
from flp2p.utils import plot_topology, build_topology, compute_weight_matrix

In [2]:
from hydra import compose, initialize

with initialize(version_base=None, config_path="conf", job_name="test_app"):
    cfg = compose(config_name="config")

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() and cfg.use_cuda else "cpu")
base_model = make_resnet18(cfg.model).to(device)
init_state = base_model.state_dict()

# Model + Clients
clients: List[FLClient] = []
for i in range(cfg.partition.num_clients):
    if cfg.model.name == "lenet5":
        model = LeNet5(cfg.model).to(device)
        model.load_state_dict(init_state)
    elif cfg.model.name == "resnet18":
        model = make_resnet18(cfg.model).to(device)
        model.load_state_dict(init_state)
    else:
        raise ValueError(f"Unknown model: {cfg.model.name}")
    train_loader, test_loader = None, None
    client = FLClient(
        model=model,
        device=device,
        train_loader=train_loader,
        test_loader=test_loader,
        config=cfg.client
    )
    clients.append(client)
    
    
graph = build_topology(cfg.partition.num_clients, cfg.graph, mixing_matrix=cfg.mixing_matrix, seed=cfg.seed, consensus_lr=cfg.consensus_lr)
W = compute_weight_matrix(graph, cfg.mixing_matrix)

c:\Users\GXGG8822\Documents\FLP2P\p2p\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\GXGG8822\Documents\FLP2P\p2p\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [13]:
from flp2p.utils import compute_consensus_distance
def get_flat_params(model):
    """Flatten model parameters into a single vector."""
    return torch.cat([p.data.view(-1) for p in model.parameters()])

def set_flat_params(model, flat_params):
    """Load a flat parameter vector into a model."""
    idx = 0
    for p in model.parameters():
        num_params = p.numel()
        p.data.copy_(flat_params[idx:idx + num_params].view_as(p))
        idx += num_params

N = len(clients)
for round in range(1, 0 + 1):
    with torch.no_grad():
        flat_params  = [get_flat_params(client.model) for client in clients]
        new_params = []
        
        for i in range(N):
            mixed = torch.zeros_like(flat_params[0])
            for j in range(N):
                mixed += W[i, j] * flat_params[j]
            new_params.append(mixed)
        
        # Update each client model
        for i in range(N):
            set_flat_params(clients[i].model, new_params[i])

compute_consensus_distance([get_flat_params(client.model) for client in clients])

NameError: name 'torch' is not defined